In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# Simple Binary Classifier for MNIST Dataset from pickle #

### Generate MNIST Dataset from Pickle ###

In [2]:
import numpy as np
import os
import pickle
from itertools import count
import matplotlib.pyplot as plt

#Code ffrom CNN course in as part of Part 1B Paper 8
with open('mnist.pkl', 'rb') as mnist_pickle:
    mnist = pickle.load(mnist_pickle, encoding='bytes')
    
mnist['data'] = mnist['data'].astype(np.float32) # convert the uint8s to floats
mnist['data'] /= 255 # scale to be from 0 to 1
mnist['target'] = mnist['target'].astype(np.int32) # convert the uint8s to int32s

def set_mnist_pos_neg(positive_label, negative_label):
    positive_indices = [i for i, j in zip(count(), mnist['target']) if j == positive_label]
    negative_indices = [i for i, j in zip(count(), mnist['target']) if j == negative_label]

    positive_images = mnist['data'][positive_indices]
    negative_images = mnist['data'][negative_indices]

    fig = plt.figure()
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(positive_images[0].reshape(28,28), cmap='gray', interpolation='nearest')
    ax.set_xticks([])
    ax.set_yticks([])
    ax = fig.add_subplot(1, 2, 2)
    plt.imshow(negative_images[0].reshape(28,28), cmap='gray', interpolation='nearest')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()
    
    return positive_images, negative_images

positive_images, negative_images = set_mnist_pos_neg(1, 0)


<Figure size 640x480 with 2 Axes>

In [3]:
def print_image_size_info(images):
    print("There are {0} images, each of shape {1} x {2}".format(images.shape[0],images.shape[1]**0.5,images.shape[1]**0.5))
    
print_image_size_info(positive_images)
print_image_size_info(negative_images)

There are 7877 images, each of shape 28.0 x 28.0
There are 6903 images, each of shape 28.0 x 28.0


In [4]:
#Shuffle Images
np.random.shuffle(positive_images)
np.random.shuffle(negative_images)

#Generate x_train and y_train (70% of all data for the training set)
x_train = np.concatenate([positive_images[:5500],negative_images[:4800]])
x_train = x_train.reshape(-1,1,28,28)
y_train = np.concatenate([np.ones(5500),np.zeros(4800)])

#Shuffle both x_train and y_train together. This step is important for the neural network to learn well!
permutation = np.random.permutation(x_train.shape[0])
x_train = x_train[permutation]
y_train = y_train[permutation]

#Convert np arrays into torch tensors
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

#Generate x_test and y_test (30% of all data for the test set)
x_test = np.concatenate([positive_images[5500:],negative_images[4800:]])
x_test = x_test.reshape(-1,1,28,28)
y_test = np.concatenate([np.ones(2377),np.zeros(2103)])

#Convert np arrays into torch tensors
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)

print(x_train.shape)
print(y_train.shape)

torch.Size([10300, 1, 28, 28])
torch.Size([10300])


### Define the Network ###


In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 4 * 4, 120)     #Remember to change this based on the input layer dimensions!
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


### Run neural net ###

In [9]:
i = 0

inputs = x_train[i*4:(i+1)*4,:,:,:]
labels = y_train[i*4:(i+1)*4]

labels = labels.long()

outputs = net(inputs)
loss = loss_fn(outputs, labels)

print(labels)
print(inputs)


tensor([ 0,  1,  1,  1])
tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
 

In [7]:
import torch.optim as optim

#Define loss function and optimizer (SGD with momentum)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [52]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(2575):
        # get the inputs
        
        inputs = x_train[i*4:(i+1)*4,:,:,:]
        labels = y_train[i*4:(i+1)*4]
        
        labels = labels.long()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss
        if i % 500 == 1999:    # print every 500 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

KeyboardInterrupt: 

### Evaluate neural net for MNIST dataset from pickle ###

In [8]:
correct = 0
total = 0
with torch.no_grad():
    for i in range(1120):
        images = x_test[i*4:(i+1)*4,:,:,:]
        labels = y_test[i*4:(i+1)*4]
        
        labels = labels.long()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 99 %


# ConvNN for full MNIST Dataset from torchvision dataset #

### Generate Dataset for MNIST from torchvision dataset ###

In [39]:
import torchvision

import torchvision.transforms as transforms


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

### Define the Network ###

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 4 * 4, 120)     #Remember to change this based on the input layer dimensions!
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### Run neural net ###

In [41]:
import torch.optim as optim

#Define loss function and optimizer (SGD with momentum)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [42]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        #inputs,labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print(labels)
print(inputs)

[1,  2000] loss: 1.348
[1,  4000] loss: 0.213
[1,  6000] loss: 0.143
[1,  8000] loss: 0.126
[1, 10000] loss: 0.100
[1, 12000] loss: 0.096
[1, 14000] loss: 0.076
[2,  2000] loss: 0.070
[2,  4000] loss: 0.067
[2,  6000] loss: 0.073
[2,  8000] loss: 0.050
[2, 10000] loss: 0.060
[2, 12000] loss: 0.062
[2, 14000] loss: 0.048
tensor([ 0,  4,  6,  0])
tensor([[[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]],


        [[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.00

tensor([ 0,  4,  6,  0])


### Evaluate neural net for MNIST dataset from torchvision ###

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 98 %
